<a href="https://colab.research.google.com/github/wakkobr/DIONTT/blob/main/sistema_bancario_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sistema Bancário em Python
# Autor: Gabriel Pessine
# Data: 21/09/2024
# Descrição: Sistema Bancário simples em Python, para o desafio do DIO/NTT - Modulo Sintaxe Basica com Pyhton

import re
from datetime import datetime

# Variaveis globais

saldo = 0            # Saldo atual da conta
lim_dia_saque = 3    # Limite máximo de saques por dia
val_max_saque = 500  # Valor máximo por saque
cont_saque = 0       # Contador de saques realizados no dia
numero_conta = 1     # Contador para o número da conta

# Listas globais

usuarios = []        # Lista de usuários
contas = []          # Lista de contas
depositos = []       # Lista para armazenar os depósitos
saques = []          # Lista para armazenar os saques


# Função DEPOSITAR

def depositar(conta, saldo, valor, extrato):
    # Realiza um depósito na conta.
    # Var:
    #    conta (float): Conta na qual as ops serão efetuadas
    #    saldo (float): Saldo atual da conta.
    #    valor (float): Valor do depósito.
    #   extrato (list): Lista com o histórico de transações.
    #
    # Retorna:
    #    tuple: Tupla contendo o saldo atualizado e o extrato atualizado.

    if valor > 0:
        conta['saldo'] += valor
        conta['extrato'].append(f"Depósito: R$ {valor:.2f}")
        print("\nDepósito de R$ {:.2f} realizado com sucesso.".format(valor))
    else:
        print("\nValor inválido para depósito.")

    return saldo, extrato

# Função SACAR

def sacar(conta, valor, saldo, extrato):
    # Realiza um saque na conta.
    # Var:
    #    conta (float): Conta na qual as ops serão efetuadas
    #    valor (float): Valor do saque.
    #    saldo (float): Saldo atual da conta.
    #    extrato (list): Lista com o histórico de transações.
    #
    # Retorna:
    #    tuple: Tupla contendo o saldo atualizado e o extrato atualizado.

    global saldo, cont_saque
    if cont_saque < lim_dia_saque and valor <= val_max_saque and saldo >= valor and valor > 0:
        conta['saldo'] -= valor
        conta['cont_saque'] += 1   # Incrementa contador de saques, para checar o limite depois
        print("\n==================================================")
        print(f"Saque de R$ {valor:.2f} realizado com sucesso.")
        print("==================================================")
    else:
        if conta['cont_saque'] >= lim_dia_saque:
            print("\n==================================================")
            print(f"Você atingiu o limite máximo de {lim_dia_saque} saques diários.")
            print("==================================================")
        elif valor > val_max_saque:
            print("\n=============================================================================")
            print(f"O valor do saque excede o limite máximo por operação, que é de R$ {val_max_saque:.2f}.")
            print("=============================================================================")
        elif valor <= 0:
            print("\n==================================================")
            print("Valor inválido, deve ser maior que zero.")
            print("==================================================")
        else:
            print("\n==================================================")
            print(f"Saldo insuficiente. Seu saldo atual é de R$ {conta['saldo']:.2f}.")
            print("==================================================")
        return saldo, extrato

# Função EXTRATO

def extrato(conta, saldo, *, extrato):
    # Imprime o extrato da conta.
    # Var:
    #    conta (float): Conta na qual as ops serão efetuadas
    #    saldo (float): Saldo atual da conta.
    #    extrato (list): Lista com o histórico de transações.

    print("\n==================================================")
    print(f"Extrato da conta {conta['numero']}:\n")
    print("==================================================\n")

    if not conta['extrato']:
        print("Não foram realizadas movimentações nessa conta.")
    else:
        for conta['valor'] in conta['extrato']:
            print(f"Valor: R$ {conta['valor']:.2f}")
        print(f"\nSaldo atual: R$ {conta['saldo']:.2f}")

    print("==================================================")

# Função CRIAR_USUARIO

def criar_usuario(nome, data_nascimento, cpf, endereco):
    # Cria um novo usuário e o adiciona à lista de usuários.
    # Var:
    #    nome (str): Nome completo do usuário.
    #    data_nascimento (str): Data de nascimento no formato 'DD/MM/AAAA'.
    #    cpf (str): Número do CPF sem pontuação.
    #    endereco (str): Endereço no formato 'logradouro - numero - bairro - cidade/sigla_estado'.
    #
    # Retorna:
    #    bool: True se o usuário foi criado com sucesso, False caso contrário.

    # Validações
    if not nome:
        print("O nome não pode ser vazio.")
        return False
    else:
        if len(nome.split()) < 2:
        print("O nome deve conter pelo menos duas palavras (nome e sobrenome).")
        return False

    if not data_nascimento:
        print("A data de nascimento não pode ser vazia.")
        return False

    # Validação da data de nascimento
    try:
        datetime.strptime(data_nascimento, '%d/%m/%Y')
    except ValueError:
        print("Data de nascimento inválida. Utilize o formato DD/MM/AAAA.")
        return False

    # Remove pontos e traços do CPF e valida
    cpf = cpf.replace('.', '').replace('-', '')
    if not validar_cpf(cpf):
        print("CPF inválido.")
        return False

    if not endereco:
        print("O endereço não pode ser vazio.")
        return False

    # Validação do endereço (expressão regular)
    padrao_endereco = r"^\w+ \d+ \w+ \w+/\w{2}$"
    if not re.match(padrao_endereco, endereco):
        print("Endereço inválido. Utilize o formato 'Logradouro - Num - Bairro - Cidade/Sigla do estado'.")
        return False

    # Cria o usuário
    novo_usuario = {
        'nome': nome,
        'data_nascimento': data_nascimento,
        'cpf': cpf,
        'endereco': endereco
    }
    usuarios.append(novo_usuario)
    return True

# Função VALIDAR_CPF

def validar_cpf(cpf):

    # Verifica se o CPF possui 11 dígitos
    if len(cpf) != 11:
        return False

    # Verifica se todos os caracteres são dígitos
    if not cpf.isdigit():
        return False

    return True

# Função CRIAR_CC

def criar_cc(usuario, agencia="0001", saldo_inicial=saldo):
    # Cria uma nova conta corrente.
    #
    # Var:
    #    usuario (dict): Dicionário representando o usuário.
    #    agencia (str, opcional): Número da agência. Defaults to "0001".
    #    saldo_inicial (float, opcional): Saldo inicial da conta. Defaults to 0.
    #
    # Retorna:
    #    bool: True se a conta foi criada com sucesso, False caso contrário.

    global numero_conta

    # Verifica se o usuário existe
    if usuario not in usuarios:
        print("Usuário não encontrado.")
        return False

    # Cria a nova conta
    nova_conta = {
        "agencia": agencia,
        "numero": numero_conta,
        "usuario": usuario,
        "saldo": saldo_inicial,
        "saques": 0
    }
    contas.append(nova_conta)
    numero_conta += 1

    return True

# Função CONSULTAR_SALDO

def consultar_saldo(conta):
    print(f"Saldo da conta {conta['numero']}: R$ {conta['saldo']:.2f}")

# Função LISTAR_CONTAS_USUARIO

def listar_contas_usuario(usuario):
    if usuario['contas']:
        print(f"Contas do usuário {usuario['nome']}:")
        for conta in usuario['contas']:
            print(f"Agência: {conta['agencia']} - Número: {conta['numero']}")
    else:
        print("O usuário não possui contas.")

# Função BUSCAR_USUARIO_POR_CPF

def buscar_usuario_por_cpf(cpf):
    for usuario in usuarios:
        if usuario['cpf'] == cpf:
            return usuario
    return None

# Função BUSCAR_CONTA_POR_NUMERO

def buscar_conta_por_numero(usuario, numero_conta):
    for conta in usuario['contas']:
        if conta['numero'] == numero_conta:
            return conta
    return None

# Função MENU

def menu():
    print("=========================")
    print("Escolha a operação:")
    print("=========================")
    print("\nU - Criar Usuário\nC - Criar Conta\nO - Consultar Saldo\nL - Listar Contas\nD - Depósito\nS - Saque\nE - Extrato\nX - Sair")
    return input("\nDigite a opção desejada: ").upper()

# Programa principal

while True:
    opcao = menu()

    if opcao == 'D':                                                  #depositar
        cpf = input("Digite o CPF do usuário: ")
        usuario = buscar_usuario_por_cpf(cpf)
        if usuario:
            numero_conta = int(input("Digite o número da conta: "))
            conta = buscar_conta_por_numero(usuario, numero_conta)
            if conta:
                valor = float(input("Digite o valor a ser depositado: "))
                depositar(conta, valor)  # Passamos apenas a conta e o valor
            else:
                print("Conta não encontrada.")
        else:
            print("Usuário não encontrado.")

   elif opcao == 'S':                                                #sacar
        cpf = input("Digite o CPF do usuário: ")
        usuario = buscar_usuario_por_cpf(cpf)
        if usuario:
            numero_conta = int(input("Digite o número da conta: "))
            conta = buscar_conta_por_numero(usuario, numero_conta)
            if conta:
                valor = float(input("Digite o valor a ser sacado: "))
                sacar(conta, valor)
            else:
                print("Conta não encontrada.")
        else:
            print("Usuário não encontrado.")

    elif opcao == 'E':                                                #extrato
        cpf = input("Digite o CPF do usuário: ")
        usuario = buscar_usuario_por_cpf(cpf)
        if usuario:
            numero_conta = int(input("Digite o número da conta: "))
            conta = buscar_conta_por_numero(usuario, numero_conta)
            if conta:
                extrato(conta)
            else:
                print("Conta não encontrada.")
        else:
            print("Usuário não encontrado.")

    elif opcao == 'U'                                                 #criar_usuario
        nome = input("Digite o nome completo: ")
        data_nascimento = input("Digite a data de nascimento (DD/MM/AAAA): ")
        cpf = input("Digite o CPF: ")
        endereco = input("Digite o endereço: ")
        novo_usuario = criar_usuario(nome, data_nascimento, cpf, endereco)
        if novo_usuario:
            print("Usuário criado com sucesso!")
        else:
            print("Erro ao criar usuário.")

    elif opcao == 'C'                                                 #criar_cc
        cpf = input("Digite o CPF do usuário: ")
        usuario = buscar_usuario_por_cpf(cpf)
        if usuario:
            agencia = input("Digite o número da agência: ")
            saldo_inicial = float(input("Digite o saldo inicial: "))
            nova_conta = criar_conta(usuario, agencia, saldo_inicial)
            print("Conta criada com sucesso!")
        else:
            print("Usuário não encontrado.")

    elif opcao == 'O'                                                 #consultar_saldo
        cpf = input("Digite o CPF do usuário: ")
        usuario = buscar_usuario_por_cpf(cpf)
        if usuario:
            numero_conta = int(input("Digite o número da conta: "))
            conta = buscar_conta_por_numero(usuario, numero_conta)
            if conta:
                consultar_saldo(conta)
            else:
                print("Conta não encontrada.")
        else:
            print("Usuário não encontrado.")

    elif opcao == 'L'                                                 #listar_contas_usuario
        cpf = input("Digite o CPF do usuário: ")
        usuario = buscar_usuario_por_cpf(cpf)
        if usuario:
            listar_contas_usuario(usuario)
        else:
            print("Usuário não encontrado.")

    elif opcao == 'X':                                                #sair
        break

    else:
        print("\nOpção inválida.")

Escolha a operação:

1 - Depósito
2 - Saque
3 - Extrato
4 - Sair

Digite a opção desejada: 1

Digite o valor que quer depositar: 100

Depósito de R$ 100.00 realizado com sucesso.
Escolha a operação:

1 - Depósito
2 - Saque
3 - Extrato
4 - Sair

Digite a opção desejada: 1

Digite o valor que quer depositar: 150

Depósito de R$ 150.00 realizado com sucesso.
Escolha a operação:

1 - Depósito
2 - Saque
3 - Extrato
4 - Sair

Digite a opção desejada: 2
Digite o valor que quer sacar: 50

Saque de R$ 50.00 realizado com sucesso.
Escolha a operação:

1 - Depósito
2 - Saque
3 - Extrato
4 - Sair

Digite a opção desejada: 1

Digite o valor que quer depositar: 125

Depósito de R$ 125.00 realizado com sucesso.
Escolha a operação:

1 - Depósito
2 - Saque
3 - Extrato
4 - Sair

Digite a opção desejada: 2
Digite o valor que quer sacar: 80

Saque de R$ 80.00 realizado com sucesso.
Escolha a operação:

1 - Depósito
2 - Saque
3 - Extrato
4 - Sair

Digite a opção desejada: 3

Extrato:

Depósito: R$ 100.00
D